### Niwot Ridge, Colorado, USA

**Station name:** nwot

**Location:** [Niwot Ridge LTER](https://nwt.lternet.edu)

**Archive:** [EarthScope Consortium](http://www.earthscope.org)

**Ellipsoidal Coordinates:**

- Latitude: 40.05539 

- Longitude: -105.59053

- Height(m): 3522.729 

[EarthScope station page](https://www.unavco.org/instrumentation/networks/status/nota/overview/NWOT)



<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/NWOT.jpg" width=500/>


In [ ]:
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

**Fast Mode:** 
this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(
    value=True, description="Fast Mode", disabled=False, indent=False
)
weekly

Deselect the Fast Mode checkbox to deactivate fast mode.

### Data Summary
Station nwot was originally installed/designed to support GPS reflections research. The site was hosted by the Niwot Ridge LTER.

nwot was made to be taller than the typical geodetic antenna so that it would not be buried by snow. It is about 3 meters above the bare soil surface. Because it was installed to support testing GPS reflections, nwot has always tracked L2C.

For this example, we will focus on the data between 2009-2015.

### Imports

### Make a SNR File and run quickLook

We will start by making a SNR file. The main archive for this dataset only provides the high-quality L2C data in the highrate (1-sec) area. We do not need this sample rate for GPS reflectometry, we will use the "special" archive option where the 1-sec data have been decimated to 15 seconds:

In [ ]:
station = "nwot"
year = 2014
doy = 270

In [ ]:
rinex2snr(station, year, doy, archive="special")

# if for some reason the above command does not work - uncomment below and use that instead:
# rinex2snr(station, year, doy, archive='unavco', rate=high, dec=15)

Now let's use this `quickLook` command to get a sense of the quality of the 
reflector height (RH) retrievals. 

First L1:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

These periodograms are a bit busy in the low RH area. But there are 
nice strong peaks in a few of the quadrants. So we can see from this that there may be some azimuth ranges that we will want to remove for teh analysis.

Now try L2:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=2)

This plot will have both L2C and non-L2C. There are failed tracks in the gray that are the non-L2C signals.

See L2C only:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=20)

Those failed tracks are now gone and we will use L2C and not all of L2.

### Make multiple years of SNR files 

We are going to look at the data from installation (Fall 2009) through Spring 2015.

In [ ]:
rinex2snr(
    station,
    year=2009,
    doy=240,
    year_end=2015,
    doy_end=120,
    archive="special",
    weekly=weekly.value,
)

### Run gnssir for multiple years
We will use a peak to noise of about 3 and a spectral peak amplitude of 8. We'll also use the southern quadrants (azimuths 90 through 270). We should note that since L5 was not tracked at this site, we will not include it in the json file. Well use a minimum elevation angle of 7 degrees because this particular receiver had a limit on the number of satellites it could track.

Save your gnssir strategy:

In [ ]:
make_gnssir_input(
    station, e1=7, peak2noise=3.2, ampl=8, azlist2=[90, 270], frlist=[1, 20]
)

Now we will set our azimuth range and the frequencies manually.

Run gnssir for the years 2009-2015:

In [ ]:
gnssir(station, year=2009, doy=1, doy_end=366, year_end=2015)

### Compute daily average RH values
Now we will use the daily_avg utility to compute RH for each day. A median filter of 0.25 meter is used to eliminate large outliers and a minimum number of tracks will be set to 10. This is relatively low because of the small number of L2C transmitting satellites in the early years of the dataset.

In [ ]:
daily_avg(
    station,
    medfilter=0.25,
    ReqTracks=10,
    year1=2009,
    year2=2015,
    txtfile="nwot_dailyRH.txt",
)

### Validation

The GPS site at Niwot Ridge was installed because there was a long-standing experiment 
for measuring snow depth. We therefore have a way to assess
accuracy. We download the *in situ* data from 
the [Niwot Ridge facility](https://portal.edirepository.org/nis/mapbrowse?scope=knb-lter-nwt&identifier=34) and will compare to pole 16.

<code>snow depth = RH_baresoil - RH</code>

We will make an estimate of the bare soil reflector height by taking an average of the reflector heights from august to mid september.

We will then download the data and plot it over the same time period that we analyzed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

# in situ data for Niwot Ridge LTER
# https://portal.edirepository.org/nis/mapbrowse?scope=knb-lter-nwt&identifier=34
insitufile = "../../../docs/use_cases/saddsnow.dw.data.csv"
# save the date, pole#, and snow depth columns (cm)
insitu = np.loadtxt(
    insitufile, skiprows=1, delimiter=",", dtype="str", usecols=(3, 2, 4)
)
# pick out pole 16, which is closest to nwot GPS site
ij = insitu[:, 1] == "16"
insitu = insitu[ij, :]

date = insitu[:, 0]  # date string
sval = insitu[:, 2]  # snow depth as string

# arrays for the in insitu answers
obst = np.empty(shape=[0, 1])
snow = np.empty(shape=[0, 1])
for i in range(0, len(insitu)):
    y = int(date[i][0:4])
    m = int(date[i][5:7])
    d = int(date[i][8:10])
    s = float(sval[i]) / 100
    obst = np.append(obst, datetime(year=y, month=m, day=d))
    snow = np.append(snow, s)

# read in the daily average RH file
gpsfile = f"{refl_code_loc}/Files/nwot/nwot_dailyRH.txt"
gps = np.loadtxt(gpsfile, comments="%")

# going to use august and mid-september to determine "no snow level"
# for other sites, you might be able to use all of september ...
# doy 213 through doy 258
# RH is stored in column 2, doy is in column 1
ii = (gps[:, 1] >= 213) & (gps[:, 1] <= 258)
noSnowRH = np.mean(gps[ii, 2])
print("No snow RH value: ", "{0:7.3f}".format(noSnowRH), "(m)")

# make a datetime array for plotting the gps results
gobst = np.empty(shape=[0, 1])
for i in range(0, len(gps)):
    gobst = np.append(
        gobst, datetime(year=int(gps[i, 0]), month=int(gps[i, 4]), day=int(gps[i, 5]))
    )

snowAccum = noSnowRH - gps[:, 2]

plt.figure()
plt.plot(gobst, snowAccum, "b.", label="GPS-IR")
plt.plot(obst, snow, "ro", label="Manual - Pole16", markersize=6)
plt.legend(loc="upper left")
plt.title("Snow Depth, Niwot Ridge LTER Saddle")
plt.xlabel("Years")
plt.ylabel("meters")
plt.grid()
left = datetime(year=2009, month=9, day=1)
right = datetime(year=2015, month=5, day=1)
plt.xlim((left, right))
plt.ylim((-0.05, 3.25))
plt.show()
# GPS receiver failed in spring 2015

Citation for data:

Walker, S., J. Morse, and Niwot Ridge LTER. 2020. Snow depth data for saddle snowfence, 1992 - ongoing. ver 11. Environmental Data Initiative. [https://doi.org/10.6073/pasta/abf23758a2e5194aded95bd18c8cbf27] (Accessed 2021-10-19).


This is one of those cases where we would want to note that pole measurements are not representative of the same
footprint as the GPS measurements. As GPS measurements cover an average over a much larger region.

We do not continue with a more quantitative comparison for this demo as there are at least two 
publications in journals and a PhD Dissertation:

[Gutmann, E., K. M. Larson, M. Williams, F.G. Nievinski, and V. Zavorotny, 
Snow measurement by GPS interferometric reflectometry: an evaluation at Niwot Ridge, Colorado, Hydrologic Processes, Vol. 26, 2951-2961, 2012](https://www.kristinelarson.net/wp-content/uploads/2015/10/GutmannEtAl_2012.pdf)

[Nievinski, F.G. and K.M. Larson, Inverse Modeling of GPS Multipath for Snow Depth Estimation, Part II: Application and Validation, IEEE TGRS, Vol. 52(10), 6564-6573, doi:10.1109/TGRS.2013.2297688, 2014](https://www.kristinelarson.net/wp-content/uploads/2015/10/felipe_inv2_revised.pdf)

[Nievinski, F.G., Ph.D. Dissertation, University of Colorado, 2013](https://www.researchgate.net/publication/258848060_Forward_and_Inverse_Modeling_of_GPS_Multipath_for_Snow_Monitoring)